<a href="https://colab.research.google.com/github/JHyunjun/TF2.0_AE_LSTM/blob/main/LSTMAE_tf2_0_forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Created by Hyunjun, JANG
# limitsinx.tistory.com

import numpy as np
import pandas as pd
import tensorflow as tf
import pandas_datareader.data as pdr
import matplotlib.pyplot as plt
import datetime
tf.random.set_seed(1)


#Data Load
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/LSTM/test_v.txt')
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/LSTM/train_v.txt')
train_data = df_train.iloc[:,:]
test_data = df_test.iloc[:,:]
print(train_data)
print(test_data)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Normalization
def MinMaxScaler(data):
  denom = np.max(data,0) - np.min(data,0)
  nume = data-np.min(data,0)
  return nume/denom

def back_MinMax(data,max,min):
  diff = max-min
  back = data * diff + min
  return back

In [ ]:
#Hyper parameters
seqLength = 60
dataDim = df_test.shape[1]
outputDim = 1
hiddenDim = 50
lr = 0.002
iterations = 100
batch_size = 16

trainSize = int(len(train_data))
trainSet = train_data[0:trainSize]
testSize = int(len(test_data))
testSet = test_data[0:testSize]
trainSet_for_result = trainSet
testSet_for_result = testSet

trainSet_max = np.max(trainSet)
trainSet_min = np.min(trainSet)
testSet_max = np.max(testSet)
testSet_min = np.min(testSet)
trainSet_max = (trainSet_max[dataDim-1])
trainSet_min = (trainSet_min[dataDim-1])
testSet_max = (testSet_max[dataDim-1])
testSet_min = (testSet_min[dataDim-1])


trainSet = MinMaxScaler(trainSet)
testSet = MinMaxScaler(testSet)

print(trainSet.shape) # (5807,5)
print(testSet.shape) # (10882,5)

forecasting = 600

In [ ]:
#Data Windowing
def sliding_window(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range(0, len(time_series) - seq_length - forecasting): 
        print(i)
        _x = time_series.iloc[i:i + seq_length, : ]
        _y = time_series.iloc[i + seq_length + forecasting, : ]  
        print(_x, "->", _y)
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

trainX, trainY = sliding_window(trainSet, seqLength)
testX, testY = sliding_window(testSet, seqLength)


In [ ]:
# Stacked LSTM Modeling
# Return Sequences is very important for stacking
# This means weather output is Sequence of this layer or all of sequences
# Activation function "tanh" : Glorot normal initializer
# Activation function "reLU" : He Initiailizer
from tensorflow import keras
from tensorflow.keras import layers

initializer = tf.keras.initializers.GlorotNormal()
input_dim = trainSet.shape[1]
model = keras.Sequential()
model.add(layers.LSTM(units = hiddenDim, activation='tanh',input_shape = (seqLength,input_dim), return_sequences = True, kernel_initializer = initializer))
model.add(layers.LSTM(units = hiddenDim*2, activation = 'tanh',return_sequences = True, kernel_initializer = initializer))
model.add(layers.LSTM(units = hiddenDim*4, activation = 'tanh',return_sequences = True, kernel_initializer = initializer))
model.add(layers.LSTM(units = hiddenDim*4, activation = 'tanh',return_sequences = True, kernel_initializer = initializer))
model.add(layers.LSTM(units = hiddenDim*2, activation = 'tanh',return_sequences = False, kernel_initializer = initializer))
model.add(layers.Dense(input_dim, activation='linear'))
adam = keras.optimizers.Adam(lr = lr)
model.compile(optimizer=adam, loss='mse')
model.summary()


In [ ]:
# Training
#testSet_LSTM = testSet[seqLength:,:]
#print(testSet_LSTM.shape)
print("train X : ",trainX.shape)
print("trainSet : ",trainSet.shape)
print("trainY : ", trainY.shape)
print("testSet : ",testSet.shape)
trainSet_LSTM = trainY[:]
print("trainSet_LSTM : ",trainSet_LSTM.shape)
hist = model.fit(trainX, trainSet_LSTM, epochs = iterations, batch_size = batch_size)

In [ ]:
# Evaluate Testset
testSet_LSTM = testY
res = model.evaluate(testX, testSet_LSTM, batch_size = batch_size)
print(res)
#res_batch_size1 = model.evaluate(testX, testY, batch_size = 1) #Very Slow


In [ ]:
# Estimating
xhat = testX
yhat = model.predict(xhat)
testY = testY.to_numpy()
print("test Y : ", testY, testY.shape,type(testY))
print("yhat : ",yhat, yhat.shape,type(yhat))
print("RMSE : {}".format(np.sqrt(np.average((yhat - testY)**2))))

In [ ]:
estimated = back_MinMax(yhat,testSet_max,testSet_min) 
estimated= estimated[:,-1]
print("Estimated : ",estimated)
real = testSet_for_result.iloc[seqLength + forecasting:,-1] # estimated + seqLength
real = real.to_numpy()

print("REAL : ",real)
plt.figure()
plt.plot(estimated[:len(estimated)],label = "Estimated by LSTM AE")
plt.plot(real[:len(real)], label = "Real")
plt.legend(prop = {'size':10})